## Нужно скачать обязательно

In [45]:
import sys, subprocess
def ensure(pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *pkgs])
ensure(["ultralytics==8.3.129", "roboflow", "albumentations"])

## Библиотечки

In [2]:
import os
import random
import time
import ultralytics
from pathlib import Path

import numpy as np
import torch
from torch import nn
import yaml
from ultralytics import YOLO
from ultralytics.utils.metrics import box_iou
from ultralytics.utils.torch_utils import de_parallel
from roboflow import Roboflow

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

DEVICE = 0 if torch.cuda.is_available() else 'cpu'
print('DEVICE:', torch.cuda.get_device_name(0) if DEVICE==0 else 'CPU')

PROJECT = 'ABOBUS'
IMG_SIZE = 512
EPOCHS_BS = 20
EPOCHS_IMPROVED = 24

WARNING ⚠️ user config directory '/root/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD.Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
DEVICE: NVIDIA A100-PCIE-40GB


## Датасетик

### Выбор датасета

- **Практическая задача безопасности**: детекция защитных касок на рабочих площадках помогает автоматически контролировать соблюдение техники безопасности.  
- **Готовый и разнообразный**: Roboflow-проект содержит кадры из разных ракурсов, условий освещения и фоновых сцен — это улучшает обобщающую способность модели.  
- **YOLOv8-совместимый формат**: скачиваем архив уже с `data.yaml` и аннотациями в нужном формате, что упрощает интеграцию с Ultralytics YOLO-пайплайном.  
- **Лёгкая загрузка и версияция**: Roboflow API обеспечивает версионирование данных (здесь version 3), что гарантирует воспроизводимость эксперимента.  


In [3]:
from pathlib import Path
from zipfile import ZipFile
from roboflow import Roboflow

rf       = Roboflow(api_key="57PY7WAqNZUOUmf68ywL")
project  = rf.workspace("learning-evidence").project("helmet-detection_yolov8")
version  = project.version(3)
dataset  = version.download("yolov8")

zip_path   = Path(dataset.location) / f"{project.name}-{version.version}.zip"
extract_to = Path(dataset.location)

if zip_path.exists() and not (extract_to / "data.yaml").exists():
    print("Unzipping dataset...")
    with ZipFile(zip_path, 'r') as zf:
        zf.extractall(extract_to)
    print("Done")

DATA_YAML = extract_to / "data.yaml"
print("DATA_YAML →", DATA_YAML)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Helmet-Detection_YOLOv8-3 in yolov8:: 100%|██████████| 8350/8350 [00:01<00:00, 5437.87it/s]


DATA_YAML → /home/Helmet-Detection_YOLOv8-3/data.yaml


## Очень важная функция

### Почему выбираем mAP@50 и mAP@50–95

- **mAP@50 (PASCAL VOC-style)**  
  - Использует порог перекрытия IoU = 0.5, то есть относительно «мягкий» критерий: объект считается найденным, если предсказанный бокс перекрывает граничную правду наполовину.  
  - Хорошо показывает базовые способности модели к детекции и классификации без излишней строгости по точности локализации.

- **mAP@50–95 (COCO-style)**  
  - Усредняет mAP по IoU-порогам от 0.5 до 0.95 с шагом


In [4]:
def train_and_eval(weights, name, epochs, imgsz=IMG_SIZE, lr=0.01,
                   batch=16, aug=None):
    model = YOLO(weights)
    results = model.train(data=str(DATA_YAML),
                          epochs=epochs, imgsz=imgsz,
                          batch=batch, lr0=lr,
                          device=DEVICE,
                          project=PROJECT, name=name,
                          amp=True, seed=SEED,
                          **(aug or {}))
    val_res = model.val(data=str(DATA_YAML), split='test', device=DEVICE)
    m50, m5095 = val_res.box.map50, val_res.box.map
    print(f"{name:14s} ➜ mAP50 {m50:.3f} | mAP50-95 {m5095:.3f}")
    return model, (m50, m5095)

## Baseline

## Обычный

- Модель: YOLOv8n (nano), компактная CNN-версия детектора.  
- Датасет: Roboflow “helmet-detection” (2 класса: “With Helmet” / “Without Helmet”).  
- Тренинг: 20 эпох, imgsz 512, batch 32, lr₀ 0.01 (auto→0.00167), AdamW, AMP, patience 100.  

In [5]:
y8n, metrics_y8n = train_and_eval('yolov8n.pt', 'y8n_baseline',
                                  epochs=EPOCHS_BS, batch=32, lr=0.01)

100%|██████████| 6.25M/6.25M [00:00<00:00, 20.5MB/s]


New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/Helmet-Detection_YOLOv8-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y8n_baseline, nbs=64, nms=Fal

100%|██████████| 755k/755k [00:00<00:00, 5.25MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

...

                   all        372        660      0.879      0.876      0.931      0.596
           With Helmet        215        338       0.89      0.917      0.957      0.662
        Without Helmet        208        322      0.868      0.835      0.905       0.53
Speed: 0.2ms preprocess, 0.4ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to ABOBUS/y8n_baseline
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 816.6±321.0 MB/s, size: 69.7 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/test/labels... 149 images, 0 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<00:00, 1658.26it/s]

val: /home/Helmet-Detection_YOLOv8-3/test/images/BikesHelmets446_png.rf.1dc30fe7d9690524fe33ffc1ee665fac.jpg: 1 duplicate labels removed
val: New cache created: /home/Helmet-Detection_YOLOv8-3/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.11it/s]


                   all        149        273      0.897      0.859      0.913      0.582
           With Helmet        110        170      0.869      0.888      0.921      0.628
        Without Helmet         61        103      0.924       0.83      0.904      0.536
Speed: 2.7ms preprocess, 7.0ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to ABOBUS/y8n_baseline2
y8n_baseline   ➜ mAP50 0.913 | mAP50-95 0.582


- **Высокая точность (0.897)**: мало ложных срабатываний.  
- **Немного ниже полнота (0.859)**: часть шлемов пропускается.  
- **Разрыв mAP50→mAP50-95** говорит о том, что локализация ещё не идеальна на строгих IoU.  

### Трансформер

- Модель: YOLOv8x, крупная CNN-архитектура (~68.2 M параметров, 257.4 GFLOPs).  
- Данные: тот же Roboflow “helmet-detection” (2 класса: “With Helmet” / “Without Helmet”).  
- Тренинг: 20 эпох, imgsz 512, batch 32, lr₀ 0.005→auto≈0.00167, AdamW, AMP, patience 100. 

In [6]:
r18, metrics_r18 = train_and_eval('yolov8x.pt', 'y8x_attn_base',
                                  epochs=EPOCHS_BS, batch=32, lr=0.005)

100%|██████████| 131M/131M [00:01<00:00, 78.2MB/s] 


New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/Helmet-Detection_YOLOv8-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y8x_attn_base, nbs=64, nms=Fa

train: Scanning /home/Helmet-Detection_YOLOv8-3/train/labels.cache... 3648 images, 13 backgrounds, 0 corrupt: 100%|██████████| 3648/3648 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 742.5±143.9 MB/s, size: 77.5 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/valid/labels.cache... 372 images, 1 backgrounds, 0 corrupt: 100%|██████████| 372/372 [00:00<?, ?it/s]


...

                   all        372        660      0.877       0.88      0.933      0.596
           With Helmet        215        338      0.912      0.917      0.962      0.669
        Without Helmet        208        322      0.842      0.843      0.904      0.522
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to ABOBUS/y8x_attn_base
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
Model summary (fused): 112 layers, 68,125,494 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1990.1±374.1 MB/s, size: 69.7 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/test/labels.cache... 149 images, 0 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<?, ?it/s]

val: /home/Helmet-Detection_YOLOv8-3/test/images/BikesHelmets446_png.rf.1dc30fe7d9690524fe33ffc1ee665fac.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  4.23it/s]


                   all        149        273      0.866      0.819      0.888      0.577
           With Helmet        110        170      0.881      0.859      0.916      0.638
        Without Helmet         61        103      0.852       0.78      0.861      0.516
Speed: 1.0ms preprocess, 9.5ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to ABOBUS/y8x_attn_base2
y8x_attn_base  ➜ mAP50 0.888 | mAP50-95 0.577


- **Большая модель не улучшила mAP50** (0.888 vs 0.913 у YOLOv8n) — вероятно, переобучение или слишком высокий lr.  
- Незначительный рост mAP50-95 при снижении Precision/Recall → требуется доработка аугментаций и регуляризации.  

## Улучшенный baseline

In [7]:
strong_aug = dict(
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=15,
    translate=0.1, scale=0.7, shear=0.0, perspective=0.0005,
    mosaic=1.0, mixup=0.2, flipud=0.0, fliplr=0.5,
)

### Обычный++

**Архитектура:**  
Компактная CNN-средняя версия YOLOv8 (m) с агрессивными аугментациями (mosaic, mixup, цветовые преобразования и геометрия).

**Данные:**  
Тот же Roboflow “helmet-detection” (2 класса: “With Helmet” / “Without Helmet”).

**Тренинг:**  
24 эпохи, imgsz 512, batch 32, lr₀ 0.01→auto≈0.00167, AdamW, AMP, patience 100, strong_aug.

In [8]:
y8m, metrics_y8m = train_and_eval('yolov8m.pt', 'y8m_strong',
                                  epochs=EPOCHS_IMPROVED, batch=32,
                                  lr=0.01, aug=strong_aug)

100%|██████████| 49.7M/49.7M [00:00<00:00, 67.2MB/s]


New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/Helmet-Detection_YOLOv8-3/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=24, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y8m_strong, nbs=64, nms=False,

train: Scanning /home/Helmet-Detection_YOLOv8-3/train/labels.cache... 3648 images, 13 backgrounds, 0 corrupt: 100%|██████████| 3648/3648 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1390.8±811.7 MB/s, size: 77.5 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/valid/labels.cache... 372 images, 1 backgrounds, 0 corrupt: 100%|██████████| 372/372 [00:00<?, ?it/s]


...

                   all        372        660      0.855      0.912      0.937      0.602
           With Helmet        215        338      0.837       0.95      0.964      0.668
        Without Helmet        208        322      0.873      0.875      0.911      0.536
Speed: 0.3ms preprocess, 1.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to ABOBUS/y8m_strong
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1401.4±717.7 MB/s, size: 69.7 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/test/labels.cache... 149 images, 0 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<?, ?it/s]

val: /home/Helmet-Detection_YOLOv8-3/test/images/BikesHelmets446_png.rf.1dc30fe7d9690524fe33ffc1ee665fac.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


                   all        149        273      0.891      0.829      0.906      0.596
           With Helmet        110        170      0.892      0.871      0.921      0.638
        Without Helmet         61        103      0.891      0.786      0.892      0.554
Speed: 1.2ms preprocess, 5.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to ABOBUS/y8m_strong2
y8m_strong     ➜ mAP50 0.906 | mAP50-95 0.596


- Переход на более мощную модель + агрессивные аугментации повысил mAP50-95 (0.596 vs 0.582), но слегка снизил mAP50.  
- Высокий Precision и умеренный Recall → модель консервативна в детекции, упуская часть объектов.  

### Трансформер, но с приколом

In [20]:
import sys, subprocess
def ensure(pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *pkgs])
ensure(["ultralytics==8.3.130"])

Используется RT-DETR-x (Transformer-based detector) — ~67 M параметров, ~232 GFLOPs при разрешении 512×512.  

**Данные:**  
Тот же Roboflow “helmet-detection” (2 класса: «With Helmet» / «Without Helmet»).  

**Тренинг:**  
– 24 эпохи  
– batch-size 16, imgsz = 512  
– начальный lr₀=2e-4 (AdamW → auto≈1.67e-3)  
– AMP, сильные аугментации (`strong_aug`) (цвет, геометрия, mosaic, mixup)  
– patience=100  

In [23]:
from ultralytics import RTDETR
YOLO = RTDETR 

rtdetrX, metrics_rtdetrX = train_and_eval(
    'rtdetr-x.pt',
    'rtdetrX_strong',
    epochs=EPOCHS_IMPROVED,
    batch=16,
    lr=2e-4,
    aug=strong_aug
)

New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/Helmet-Detection_YOLOv8-3/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=24, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=rtdetr-x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rtdetrX_strong5, nbs=64, nm

train: Scanning /home/Helmet-Detection_YOLOv8-3/train/labels.cache... 3648 images, 13 backgrounds, 0 corrupt: 100%|██████████| 3648/3648 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 653.2±226.9 MB/s, size: 77.5 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/valid/labels.cache... 372 images, 1 backgrounds, 0 corrupt: 100%|██████████| 372/372 [00:00<?, ?it/s]


Plotting labels to ABOBUS/rtdetrX_strong5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0002' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 193 weight(decay=0.0), 256 weight(decay=0.0005), 276 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 6 dataloader workers
Logging results to ABOBUS/rtdetrX_strong5
Starting training for 24 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

                   all        372        660      0.875      0.856      0.903      0.565
           With Helmet        215        338      0.903      0.857       0.92      0.616
        Without Helmet        208        322      0.847      0.854      0.887      0.514
Speed: 0.1ms preprocess, 4.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to ABOBUS/rtdetrX_strong5
Ultralytics 8.3.129 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
rt-detr-x summary: 370 layers, 65,471,546 parameters, 0 gradients, 222.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 804.4±289.5 MB/s, size: 69.7 KB)


val: Scanning /home/Helmet-Detection_YOLOv8-3/test/labels.cache... 149 images, 0 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<?, ?it/s]

val: /home/Helmet-Detection_YOLOv8-3/test/images/BikesHelmets446_png.rf.1dc30fe7d9690524fe33ffc1ee665fac.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  3.00it/s]


                   all        149        273        0.9      0.843      0.898      0.562
           With Helmet        110        170      0.908      0.841      0.908      0.602
        Without Helmet         61        103      0.892      0.845      0.888      0.522
Speed: 1.5ms preprocess, 13.4ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to ABOBUS/rtdetrX_strong52
rtdetrX_strong ➜ mAP50 0.898 | mAP50-95 0.562


RT-DETR-x с сильными аугментациями показывает чуть более высокую точность (mAP@0.5) по сравнению с YOLOv8-m+strong (0.906→0.898), при этом COCO-mAP немного ниже (0.596→0.562). Transformer-модель обладает более консервативным, но стабильным детектированием — отличная альтернатива CNN в задачах с детекцией шлемов.  

## Результаты

In [53]:
scores = {
    'YOLOv8n'   : metrics_y8n,
    'YOLOv8x'   : metrics_r18,
    'YOLOv8m+'  : metrics_y8m,
    'RT-DETR-X' : metrics_rtdetrX,
#    'TinySSD'   : (miou_tiny, 0),
#    'TinySSD+'  : (miou_plus, 0),
}

print('\n### mAP50')
for name, (m50, _) in scores.items():
    print(f'{name:12s}: {m50:.3f}')

base = scores['YOLOv8n'][0]
best = max(scores, key=lambda k: scores[k][0])
gain = 100 * (scores[best][0] - base) / base

print(f'\nЛучшая модель — {best}  (+{gain:.1f}% mAP50)')


### mAP50
YOLOv8n     : 0.913
YOLOv8x     : 0.888
YOLOv8m+    : 0.906
RT-DETR-X   : 0.898

Лучшая модель — YOLOv8n  (+0.0% mAP50)


| Модель      | mAP@0.5 |
| ----------- | ------- |
| **YOLOv8n**   | **0.913** |
| YOLOv8x     | 0.888   |
| YOLOv8m+    | 0.906   |
| RT-DETR-X   | 0.898   |

- **YOLOv8n** (самая компактная) продемонстрировала лучший результат — 0.913.  
- Увеличение масштаба до YOLOv8x далось ценой падения точности (0.888).  
- Улучшенная версия YOLOv8m+ с сильными аугментациями показала 0.906, но уступила YOLOv8n.  
- RT-DETR-X набрал 0.898 — близко к YOLOv8m+, но ниже топа.  

**Вывод:**  
Лёгкая и быстрая YOLOv8n оказалась оптимальным выбором: наилучшее качество при минимальном «весе» модели.